In [2]:
import os
import time
import random
import timm
import torch
import albumentations as A
import pandas as pd
import numpy as np
import torch.nn as nn
from albumentations.pytorch import ToTensorV2
from torch.optim import Adam
from torchvision import transforms
from torch.utils.data import Dataset, DataLoader
from PIL import Image
from tqdm import tqdm
from sklearn.metrics import accuracy_score, f1_score
from augraphy import *
from sklearn.model_selection import KFold
from glob import glob
import cv2

### 데이터 Augmentation 처리

크게 두가지의 데이터 augmentation을 진행
1. 테스트 데이터셋에 공통적으로 나오는 노이즈(augraphy)와 유사한 전처리를 하고 Rotation & Flip만 진행한 케이스
2. Albumentation 처리 + 

In [ ]:

def augment_data_and_update_csv(input_folder, output_folder, aug, alb, csv_file, csv_name, num_augmented_per_image=4):
    # Load the original CSV file
    original_df = pd.read_csv(csv_file)

    # Create a new DataFrame for augmented data
    augmented_df = pd.DataFrame(columns=original_df.columns)

    # Define augmentation pipeline
    aug = aug # augraphy 변환

    alb = alb # albumentation 변환
    # Get a list of all image files in the input folder
    image_files = glob(os.path.join(input_folder, '*.jpg'))

    for img_path in image_files:
        # Read the image
        img = cv2.imread(img_path)

        # Apply augmentation multiple times
        for i in tqdm(range(num_augmented_per_image)):
            augraphy = aug(img)
            augmented = alb(image=augraphy)
            augmented_img = augmented['image']

            # Save augmented image
            output_path = os.path.join(output_folder, f"augmented_{i}_{os.path.basename(img_path)}")
            cv2.imwrite(output_path, augmented_img)

            # Add entry to the augmented DataFrame
            augmented_entry = {
                'ID': os.path.basename(output_path),
                'target': original_df[original_df['ID'] == os.path.basename(img_path)]['target'].values[0]
            }
            augmented_df = pd.concat([augmented_df, pd.DataFrame([augmented_entry])], ignore_index=True)

    # Concatenate original and augmented DataFrames
    combined_df = pd.concat([original_df, augmented_df], ignore_index=True)

    # Save the new CSV file
    combined_df.to_csv(os.path.join(output_folder, csv_name), index=False)

input_folder = "/data/ephemeral/home/data/train"
output_folder = "/data/ephemeral/home/data/aug"
original_csv_file = "/data/ephemeral/home/data/train.csv"

# augment_data_and_update_csv(input_folder, output_folder, original_csv_file, num_augmented_per_image=4)

In [ ]:
# 1번 처리 : augraphy
paper_phase = [

    ColorPaper(
        hue_range=(0, 255),
        saturation_range=(10, 40),
        p=0.33,
    ),
            OneOf(
        [
            DelaunayTessellation(
                n_points_range=(500, 800),
                n_horizontal_points_range=(500, 800),
                n_vertical_points_range=(500, 800),
                noise_type="random",
                color_list="default",
                color_list_alternate="default",
            ),
            PatternGenerator(
                imgx=random.randint(256, 512),
                imgy=random.randint(256, 512),
                n_rotation_range=(10, 15),
                color="random",
                alpha_range=(0.25, 0.5),
            ),
            VoronoiTessellation(
                mult_range=(50, 80),
                seed=19829813472,
                num_cells_range=(500, 1000),
                noise_type="random",
                background_value=(200, 255),
            ),
        ],
        p=1.0,
    ),
    AugmentationSequence(
        [
            NoiseTexturize(
                sigma_range=(3, 10),
                turbulence_range=(2, 5),
            ),
            BrightnessTexturize(
                texturize_range=(0.9, 0.99),
                deviation=0.03,
            ),
        ],
    ),
]

post_phase = [
    VoronoiTessellation(p=0.7),
]
pipeline = AugraphyPipeline(paper_phase=paper_phase, post_phase=post_phase)


# albumentation
alb = A.Compose([
    A.HorizontalFlip(p=0.8),
    A.VerticalFlip(p=0.8),
    A.Rotate(p=0.8),
    A.Resize(224, 224)
])

csv_name = 'augmented_ljh.csv'
augment_data_and_update_csv(input_folder, output_folder, pipeline, alb, original_csv_file, csv_name=csv_name, num_augmented_per_image=6)

In [ ]:
#2번 처리


output_folder = "/data/ephemeral/home/upstage_cv/data/aug_img/sjy"

# augraphy
ink_phase = [
    Folding(p=0.6)
    ]

paper_phase = [
    ColorPaper(p=0.6),
    ColorShift(p=0.6),
    ReflectedLight(p=0.6)
]

post_phase = [
    VoronoiTessellation(p=0.6),
]
pipeline = AugraphyPipeline(ink_phase=ink_phase, paper_phase=paper_phase, post_phase=post_phase)

aug_sjy = pipeline

# albumentation
alb_sjy = A.Compose([
    A.HorizontalFlip(p=0.6),
    A.VerticalFlip(p=0.6),
    A.Rotate(p=0.6),
    A.GaussianBlur(p=0.6),
    A.RandomBrightnessContrast(p=0.6),
    A.HueSaturationValue(p=0.6),
    A.RandomGamma(p=0.6),
    A.ColorJitter(p=0.6),
    A.CoarseDropout(p=0.6),
    A.GaussNoise(p=0.6),
    A.Resize(224, 224),
    #ToTensorV2()
])

csv_name = 'augmented_sjy.csv'
augment_data_and_update_csv(input_folder, output_folder, aug_sjy, alb_sjy, original_csv_file,  csv_name=csv_name, num_augmented_per_image=6)

### 기존 데이터셋과 합치는 방식

1. 파일명 바꾸기(동일하게 augmentation 하는 과정에서 augmented라는 파일명이 앞에 붙여져서)
2. 기존 데이터셋에서 원본 트레이닝 데이터만 지우기(2번에서 이미 있기 때문에)
3. 옮기려는 데이터를 기존 데이터셋 옮기기

In [ ]:
#실제 파일명 변경하기

import os
import pandas as pd

# 이미지가 있는 폴더 경로
image_folder = 'data/aug_1'
# CSV 파일 경로
csv_file = 'data/aug_1/augmented_ljh.csv'

# 이미지 파일명 변경
for i, filename in enumerate(os.listdir(image_folder)):
    # if "augmented" not in filename and filename.endswith(".jpg"):
    #     os.remove(os.path.join(folder_path, filename))
    #     print(f"Deleted: {filename}")  # 삭제된 파일명 출력
    if filename.endswith((".png", ".jpg", ".jpeg")):  # 이미지 파일 확장자 확인
        name, ext = os.path.splitext(filename)
        new_name = f"{name}_ljh{ext}"  # 새 파일명 형식
        os.rename(os.path.join(image_folder, filename), os.path.join(image_folder, new_name))
   



In [ ]:
# CSV 파일에서 이미지 파일명 업데이트
df = pd.read_csv(csv_file)

# 파일명을 업데이트할 CSV 컬럼명, 예를 들어 'image_name'
column_name = 'ID'


# 새로운 이미지 파일명으로 업데이트
for i, row in df.iterrows():
    old_name = row[column_name]
    new_name = old_name.replace('.jpg', '_ljh.jpg')
    df.at[i, column_name] = new_name  # 파일명 업데이트, 경로 포함 업데이트 필요 시
        

df.to_csv("new_aug.csv", index = False)

In [8]:
#원본 파일 삭제하기
for i, filename in enumerate(os.listdir(image_folder)):
    if "augmented" not in filename and filename.endswith(".jpg"):
        os.remove(os.path.join(image_folder, filename))
        print(f"Deleted: {filename}")  # 삭제된 파일명 출력


In [ ]:
#파일 이동하기
import os
import shutil

# 이미지 파일이 있는 원본 폴더 경로
source_folder = 'sjy/'

# 이미지 파일을 옮길 대상 폴더 경로
destination_folder = 'data/aug_2/'


# 원본 폴더에서 각 파일을 반복 처리
for filename in os.listdir(source_folder):
    # 파일 경로 구성
    file_path = os.path.join(source_folder, filename)
    
    # 파일이 이미지인지 확인 (확장자를 통해)
    if filename.lower().endswith(('.png', '.jpg', '.jpeg', '.gif', '.bmp')):
        # 파일을 대상 폴더로 이동
        shutil.move(file_path, destination_folder)

In [ ]:
터미널에서 아래 코드 실행
!tar -czvf /Users/username/Documents/my_folder.tar.gz /Users/username/Documents/my_folder
